In [33]:
# IMPORTS
import pandas as pd 
import wandb 
import numpy as np
import matplotlib.pyplot as plt 
import os, sys
import wandb 
import time
import json
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [34]:
# GLOBALS + wandb init
ENTITY = 'tpn'
PROJECT = 'wolf_1'

api = wandb.Api()
num_runs = len(api.runs(path=f'{ENTITY}/{PROJECT}'))
finished_runs = api.runs(
    path=f'{ENTITY}/{PROJECT}', 
    filters={"state": "finished"})
print(f"""
    Entity: {ENTITY}
    Project: {PROJECT}
    Number of runs: {num_runs}
    Finished runs: {len(finished_runs)}
    """)
runs_summary = pd.DataFrame()
for run in finished_runs:
    config = run.config
    summary = run.summary
    if run.state != 'finished':
        print(f'Run {run.name} is not finished')
        continue
    time.sleep(0.01)
    metrics = pd.DataFrame([dict(summary)])
    metrics['algorithm_type'] = config['algorithm_type']
    metrics['run_id'] = run.name
    metrics['env_config'] = json.dumps(config['env_config'])
    # metrics['nprey'] = config['env_config']['nprey']
    # metrics['reward_lone'] = config['env_config']['reward_lone']
    # metrics['reward_team'] = config['env_config']['reward_team']
    metrics['config'] = json.dumps(config)
    if len(runs_summary) == 0:
        runs_summary = metrics
    else:
        runs_summary = pd.concat([runs_summary, metrics])


    Entity: tpn
    Project: wolf_1
    Number of runs: 112
    Finished runs: 80
    


Performances of ['independent', 'shared'] alogirthms, for different r_team [0.5, 0.75, 1.0, 1.25, 1.5] and different n_prey [2, 5, 10]. Map Size and Pred Vision for {PROJECT} are set to 20 and 3 respectively. 

Index(['_wandb', 'time_total_s', 'eval_episode_len', 'episode_assists_mean',
       'eval_policy_reward_mean', 'eval_predator_1_assists', '_runtime',
       'eval_assists', 'eval_episodes', 'predator_0_kills', 'predator_1_kills',
       '_timestamp', 'policy_reward_mean', 'predator_0_assists',
       'num_env_steps_sampled', '_step', 'eval_reward', 'episodes_total',
       'episode_len_mean', 'episode_kills_mean', 'predator_1_assists',
       'training_iteration', 'episode_reward_mean', 'eval_predator_0_assists',
       'algorithm_type', 'run_id', 'env_config', 'config'],
      dtype='object')


,_wandb,time_total_s,eval_episode_len,episode_assists_mean,eval_policy_reward_mean,eval_predator_1_assists,_runtime,eval_assists,eval_episodes,predator_0_kills,...,episode_len_mean,episode_kills_mean,predator_1_assists,training_iteration,episode_reward_mean,eval_predator_0_assists,algorithm_type,run_id,env_config,config
0,{'runtime': 21502},21232.555535,87.121242,0.90,{'shared_policy': 3},0.510184,21258.209673,1.043647,1031,2.58,...,85.59,4.96,0.50,24132,5.86,0.533463,shared,df1706132011,"{""npred"": 2, ""nprey"": 6, ""map_size"": 20, ""prey...","{""ray"": {""init_dashboard"": false}, ""tune"": {""t..."
0,{'runtime': 26630},26280.609367,81.788246,0.84,"{'predator_0': 3.201492537313433, 'predator_1'...",0.588619,26309.999065,0.762127,1072,3.09,...,80.44,5.42,0.66,23656,6.26,0.173507,independent,df1706150295,"{""npred"": 2, ""nprey"": 6, ""map_size"": 20, ""prey...","{""ray"": {""init_dashboard"": false}, ""tune"": {""t..."
0,{'runtime': 21163},20894.289361,60.995559,1.55,{'shared_policy': 3.5177646188008884},0.669134,20921.598820,1.322724,1351,2.71,...,64.39,5.67,0.84,19770,7.22,0.653590,shared,df1706100433,"{""npred"": 2, ""nprey"": 6, ""map_size"": 15, ""prey...","{""ray"": {""init_dashboard"": false}, ""tune"": {""t..."
0,{'runtime': 17831},17591.046837,86.996176,0.95,{'shared_policy': 2.9153919694072656},0.384321,17614.456339,0.799235,1046,2.74,...,86.97,4.97,0.56,24180,5.92,0.414914,shared,df1706112743,"{""npred"": 2, ""nprey"": 6, ""map_size"": 20, ""prey...","{""ray"": {""init_dashboard"": false}, ""tune"": {""t..."
0,{'runtime': 24214},23866.895870,54.257955,1.08,"{'predator_0': 3.197698036560596, 'predator_1'...",0.247800,23895.270699,0.960054,1477,2.63,...,60.22,5.80,0.32,18401,6.88,0.712255,independent,df1706073503,"{""npred"": 2, ""nprey"": 6, ""map_size"": 15, ""prey...","{""ray"": {""init_dashboard"": false}, ""tune"": {""t..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,{'runtime': 24164},23512.363219,58.634463,0.47,"{'predator_0': 3.2307137707281903, 'predator_1...",0.198270,23543.537302,0.392213,1387,3.23,...,62.32,5.79,0.23,17529,6.26,0.193944,independent,df1705952922,"{""npred"": 2, ""nprey"": 6, ""map_size"": 15, ""prey...","{""ray"": {""init_dashboard"": false}, ""tune"": {""t..."
0,{'runtime': 34813},34174.532431,74.532628,0.19,"{'predator_0': 2.9576719576719577, 'predator_1...",0.107584,34215.317948,0.208113,1134,2.98,...,78.50,5.53,0.11,23155,5.72,0.100529,independent,df1706005313,"{""npred"": 2, ""nprey"": 6, ""map_size"": 20, ""prey...","{""ray"": {""init_dashboard"": false}, ""tune"": {""t..."
0,{'runtime': 39230},38665.249810,79.761730,0.63,"{'predator_0': 2.979760809567617, 'predator_1'...",0.153634,38712.978581,0.670653,1087,2.51,...,77.73,5.58,0.13,23456,6.21,0.517019,independent,df1705937886,"{""npred"": 2, ""nprey"": 6, ""map_size"": 20, ""prey...","{""ray"": {""init_dashboard"": false}, ""tune"": {""t..."
0,{'runtime': 37566},36970.988977,75.757386,0.17,"{'predator_0': 3.2318710832587287, 'predator_1...",0.120859,37016.171309,0.193375,1117,3.05,...,71.42,5.70,0.06,23120,5.87,0.072516,independent,df1705926068,"{""npred"": 2, ""nprey"": 6, ""map_size"": 20, ""prey...","{""ray"": {""init_dashboard"": false}, ""tune"": {""t..."
